In [ ]:
import pickle
import json
import numpy as np
import pickle as pkl
import os
from src.utils.data_transform import *
from src.utils.data_io import *
import importlib
from src.cnn.cnn_steps import *
# from src.utils.preprocessing import *
from src.lstm.lstm_steps import *
from src.analysis.stats import *
from src.lstm.lstm_utils import *

**Environment data setup**

In [ ]:
with open("../../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)
subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

predictions_path = "../../data/cnn_predictions/corrected/complete/timestamped"
predictions = load_cnn_predictions(subject_to_indices, predictions_path)

with open("../../data/FIC/FIC.pkl", "rb") as f:
    fic = pickle.load(f)
bite_gt = fic["bite_gt"]

**Extract information on bites**

In [ ]:
bite_lengths = []
for meal in bite_gt:
    for bite in meal:
        bite_lengths.append(bite[1] - bite[0])

In [ ]:
bite_lengths = np.array(bite_lengths)

In [ ]:
print(f'Average bite length: {np.mean(bite_lengths)}')
print(f'Min bite length: {np.min(bite_lengths)}')
print(f'Max bite length: {np.max(bite_lengths)}')

In [ ]:
print(calculate_bite_stats(bite_gt))

**Create positive example bites**

In [ ]:
positive_examples = {}
for session_id in range(len(bite_gt)):
    positive_examples[session_id], counter = create_positive_example_bites(predictions[session_id+1], bite_gt[session_id])

**Create negative example bites**

In [ ]:
negative_examples = {}
for session_id in range(len(bite_gt)):
    negative_examples[session_id] = create_negative_example_bites(predictions[session_id+1], bite_gt[session_id])

**Combine and Save both**

In [ ]:
combined_sessions = {}
combined_sessions = combine_and_balance_examples(positive_examples, negative_examples, combined_sessions)
combined_sessions = shuffle_examples(combined_sessions)
training_data = [combined_sessions[session_id] for session_id in sorted(combined_sessions.keys())]

In [ ]:
save_data(training_data, "../../data/lstm_training_data/", "corrected_balanced_training_data")

**Testing**

In [ ]:
for i in range(0,21):
    print(f"True bite length: {len(bite_gt[i])}\tCalculated bite length: {len(positive_examples[i])}")

In [ ]:
true_bites_left_21 = [(bite[1]-bite[0]) for bite in bite_gt[20] if bite[1]- bite[0] <= 9]

In [ ]:
# Sessioned bite lengths
bites_per_session = {}
session_id = 0
for session in bite_gt:
    bite_lengths = []  # Initialize list to store bite lengths for the current meal
    for bite in session:
        bite_lengths.append(bite[1] - bite[0])
    bites_per_session[session_id] = bite_lengths  # Use meal ID as key
    session_id += 1  # Increment meal ID for the next meal

In [ ]:
true_bites_per_session_less_than_9 = {}
for session_id, bite_length in bites_per_session.items():
    counter = sum([1 for bite in bite_length if bite <= 9])
    true_bites_per_session_less_than_9[session_id] = counter

In [ ]:
calculated_bites_per_session_less_than_9 = {}
for session_id, windows in positive_examples.items():
    calculated_bites_per_session_less_than_9[session_id] = len(windows)

In [ ]:
bites_comparison = []

# Iterate through the session IDs in the true bites per session dictionary
for session_id, true_count in true_bites_per_session_less_than_9.items():
    calculated_count = calculated_bites_per_session_less_than_9.get(session_id, 0)
    # Append session ID, true count, and calculated count to the session data list
    bites_comparison.append([session_id, true_count, calculated_count])

# Convert the session data list into a NumPy array
bites_comparison = np.array(bites_comparison)

In [ ]:
positive_examples[17][23][0]

In [ ]:
np.vstack((bite_gt[17].T,[bite[1] - bite[0] for bite in bite_gt[17]])).T